# Score Dinosaur National Monument w Binary Model

Each green square is roughly 260m x 260m or about 2.5 times the length of a soccer of football field

![assets/DNM_ThreeClassMainCircled.jpg](assets/DNM_ThreeClassMainCircled.jpg)

Even though this model was trained on images and data from  New Meciso it predicts very well in Utah

The red circled building is the famous Dinosaur National Monument buildig housing the bones still being excavated from teh site.

The site was discovered in 1909 by paleontologist Earl Douglass of the Carnegie Museum 


![assets/DNM_Camarasaurus.jpg](assets/DNM_Camarasaurus.jpg)

Essentially, the New Mexico trained model, predicts Bones likley at the site of this building

This is based on the texture and color of the depotiional environments of sandstones and claystones of the Brushy Basin member of the Morrison Formation

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from PIL import Image
import json
import glob
import os

scratch_model = models.resnet18(pretrained=True)
num_ftrs = scratch_model.fc.in_features
classes = 3
scratch_model.fc = nn.Linear(num_ftrs, classes)
scratch_model.load_state_dict(torch.load('resnet18-Gold20220530.pt'))
#scratch_model.load_state_dict(torch.load('resnet34.pt'))
map_save = 'data/MoabDinoTrail_ThreeClassBalanced.jpg'

# Data augmentation and normalization for training
# Just normalization for validation
input_size = 224
data_dir = "data/ThreeClassManualRemove0s/"

batch_size = 64
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((input_size,input_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((input_size,input_size)),        
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}


print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cpu")
print("device: {}".format(device))


files = []
class_true = []
class_pred = []
#my_classes = ['Bone', 'NoBone']
my_classes = image_datasets['val'].classes

green = Image.new('RGBA',(224,224),(0,255,0,60))
white = Image.new('RGBA',(224,224),(255,255,255,1))
lightGreen = Image.new('RGBA',(224,224),(0,255,0,20))
black = Image.new('RGBA',(224,224),(0,0,0,1))


def DatasetSizes(dataset_ReadClassChoices):
    dataset_sizes = {x: len(dataset_ReadClassChoices[x]) for x in ['train', 'val']}
    return dataset_sizes

def scoreSingleImage(ImagePath, model, dataset_classes):
    from PIL import Image
    import torch.nn.functional as F
    from torch.autograd import Variable
    model.eval()
    #model.to(device)
    img = Image.open(ImagePath).convert('RGB') 
    x_test = data_transforms['val'](img)[:3]   #3 channels in case png bobc
    x_test.unsqueeze_(0)  # Add batch dimension
    x_test2 = Variable(x_test)
    output = model(x_test)
    class_names = dataset_classes
    predArgmax = torch.argmax(output[0]).numpy()
    confidence = F.softmax(output, dim=0)
    score = []
    score.append( class_names[predArgmax] )
    score.append( float(confidence[0][predArgmax]) )
    return score 



In [ ]:
filename = 'data/ThreeClassBalanced5000/train/2/Batwing07.png'
scoreSingleImage(filename, scratch_model, my_classes)

# Score val folder to print metrics

In [ ]:
img = Image.open('data/Moab Dino Trail 16 NoBone Area.PNG')
XR, YR = img.size
XR, YR = XR//224, YR//224
print(XR, YR)

In [ ]:
from PIL import Image
import glob
import os

pngFolder = 'data/UT_Vac3/'
for fn in glob.glob(pngFolder + '*.png'):
    img = Image.open(fn)
    r, g, b = img.convert('RGB').split()
    if g.histogram()[0] + r.histogram()[0] + b.histogram()[0] > 223*3:
        print(fn," remove: has black stripe")
        os.remove(fn, dir_fd=None)
    img.close()
   

# Score 20220514/224

Best map to score

Green square roughly 265 m x 265 m -  about 2.5 football or soccer fields long

The are in Green is a significantly smaller search are than the entire map

In [ ]:
import time
from os.path import exists

green = Image.new('RGBA',(224,224),(0,255,0,75))
white = Image.new('RGBA',(224,224),(255,255,255,1))
lightGreen = Image.new('RGBA',(224,224),(0,255,0,40))
black = Image.new('RGBA',(224,224),(0,0,0,1))

start = time.time()

for x in range(XR):
    for y in range(YR):
        filename = 'data/UT_Vac3/DNMx{:03d}y{:03d}.png'.format(x, y)
        filenameMap = 'data/UT_Vac3/224Map/DNMx{:03d}y{:03d}.png'.format(x, y)
        if exists(filename):
            img = Image.open(filename).convert('RGB')
            try: 
                pred = scoreSingleImage(filename, scratch_model, my_classes)[0]
                print(pred, filename)
                if pred[0] == '2':
                    Image.alpha_composite(img.convert("RGBA"), green).save(filenameMap)
                elif pred[0] == '1': 
                    Image.alpha_composite(img.convert("RGBA"), lightGreen).save(filenameMap)
                else:
                    Image.alpha_composite(img.convert("RGBA"), black).save(filenameMap)
                class_pred.append(pred[0])
                class_true.append(filename.split('/')[-2])
            except:
                print ("Problem", x, y, filename)
print("Scoring time elapsed: ", time.time() - start) 


In [ ]:
scoreSingleImage(filename, scratch_model, my_classes)[0]


# Merge Dinosaur National Monument merged 20220514/224

![assets/Re-assembleTilesIntoMap.PNG](assets/Re-assembleTilesIntoMap.PNG)

In [ ]:
from PIL import Image
xblock = XR
yblock = YR
dst = Image.new('RGB', ((xblock - 1)*224, (yblock - 1)*224))
for x in range(xblock):
    for y in range(yblock):
        path = 'data/UT_Vac3/224Map/DNMx{:03d}y{:03d}.png'.format(x,y)
        if exists(path):
            img = Image.open(path)
            dst.paste(img, (x*224, y*224))
            img.close()
dst.save(map_save)
print(map_save)
print("Done!")